In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = 'true'

os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'

os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

# Part 10: Logical and Semantic Routing

Logical routing

In [23]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

# Base Model
class RouteQuery(BaseModel):
    """ Route a user query to the most relevant datasource"""
    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant to their question"
    )

# LLM with Function call
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to the appropriate data source.
Based on the progamming language the question is referring to, route it to the relevant data source.
Your answer should be the language + '_docs' where language is the programming language the question is referring to."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}")
    ]
)

# Define router
router = prompt | structured_llm

In [24]:
question = """Why doesn't the following code work:
from langchain_core import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [25]:
result

In [7]:
result.datasource

NoneType

In [6]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### logic here
        return "chain for python docs"
    elif "js_docs" in result.datasource.lower():
        ### logic here
        return "chain for js docs"
    else:
        ### logic here
        return "chain for golang docs"
    
from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [10]:
full_chain.invoke({"question": question})

AttributeError: 'NoneType' object has no attribute 'datasource'

Semantic routing

In [11]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor.\
    You are great at answering questions about physics in a concise and easy to understand manner. \
    When you don't know the answer to a question you admit that you don't know. \
        
    Here is a question:
    {query}"""

math_template = """You are a very good mathematician. You are great at answering maths questions. \
    you are good because you are able to break down hard problems into their component parts, \
    answer the component parts and then put them back together to answer the broader question. 
    
    Here is a question:
    {query}"""

# Embed prompts
embeddings = GoogleGenerativeAIEmbeddings(model= "models/embedding-001", google_api_key=os.getenv("GEMINI_API_KEY"))
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route questions to prompt
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region of spacetime where gravity is so strong that nothing, not even light, can escape. It's formed when a massive star collapses at the end of its life. 

Imagine a giant star, many times bigger than our sun, running out of fuel.  Without the outward pressure from nuclear fusion, gravity takes over, crushing the star's core into an incredibly dense point called a singularity.  The intense gravity around this singularity creates a region of space where the escape velocity exceeds the speed of light.  This is the black hole.

Here's a simple analogy:  Think of a bowling ball placed on a trampoline. It creates a dip in the fabric. Now imagine a much heavier object, like a cannonball. It creates a much deeper dip.  A black hole is like that cannonball, creating a deep, inescapable "dip" in the fabric of spacetime. 

While we can't see black holes directly, we can observe their effects on surrounding matter and light.  This is how we know they exist. 



# Part 11: Query structuring for metadata filters

In [12]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://youtu.be/sVcwVQRHIc8",
    add_video_info=True
).load()

docs[0].metadata

{'source': 'sVcwVQRHIc8',
 'title': 'Learn RAG From Scratch – Python AI Tutorial from a LangChain Engineer',
 'description': 'Unknown',
 'view_count': 334689,
 'thumbnail_url': 'https://i.ytimg.com/vi/sVcwVQRHIc8/hq720.jpg',
 'publish_date': '2024-04-17 00:00:00',
 'length': 9191,
 'author': 'freeCodeCamp.org'}

Let's assume we've built an index that:

    1. Allows us to perform unstructured search over the 'contents' and 'title' of each document
    2. And to user range filtering on 'view_count', 'publication date', and 'length'

We want to convert natural language into structured search queries.

We can define a schema for structured search queries

In [13]:
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field

class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts."
    )

    title_search: str = Field(
        ...,
        description= (
            "Alternate version of the content_search query to apply to video titles."
        )
    )

    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified."
    )

    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, inclusive. Only use if explicitly specified."
    )

    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, exclusive. Only use if explicitly specified."
    )

    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified."
    )

    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified."
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(self.__fields__[field], "default", None):
                
                print(f"{field}: {getattr(self, field)}")

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

system = """You are an expert at converting user questions into database queries. \
    You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
    Given a question, return a database query optimized to retrieve the most relevant results.
    
    If there are any acronyms or words you are not familiar with, do not try to rephrase them."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}")
    ]
)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.getenv("GEMINI_API_KEY"), temperature=0)
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyser = prompt | structured_llm

In [15]:
query_analyser.invoke({"question": "rag from scratch"}).pretty_print()

content_search: rag from scratch
title_search: rag from scratch


In [16]:
query_analyser.invoke(
    {"question": "videos on chat langchain published in 2023"}
).pretty_print()

content_search: chat langchain
title_search: chat langchain
earliest_publish_date: 2023-01-01


In [17]:
query_analyser.invoke(
    {"question": "how to use multimodal models in an agent, only videos under 5 minutes"}
).pretty_print()

content_search: multimodal models agent
title_search: multimodal models agent
max_length_sec: 300
